<a href="https://colab.research.google.com/github/Ailton3112/glp/blob/main/RevGas_NPD_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NPD Prediction

* Sources
  - [Using machine learning to predict the next purchase date for an individual retail customer](http://www.scielo.org.za/scielo.php?script=sci_arttext&pid=S2224-78902020000300008)
  - [Predicting Next Purchase Day](https://towardsdatascience.com/predicting-next-purchase-day-15fae5548027)
  - [Using Machine Learning to Predict Customers’ Next Purchase Day](https://towardsdatascience.com/using-machine-learning-to-predict-customers-next-purchase-day-7895ad49b4db)

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression


In [ ]:
#!pip3 install google-colab


#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#dataset = pd.read_csv('/content/drive/MyDrive/dataset/produtos_clientes.csv',delimiter = ',',low_memory=False)

dataset = pd.read_csv('/Users/ailtonrodrigues/Desktop/REVGAS/input/produtos_clientes.csv',delimiter = ',',low_memory=False)

dataset.head()

,client_id,quantity_product,product,value_product,payments,payments_values,total,date,city,state
0,29293,1,GLP 13KG,45.00,Dinheiro,45.00,45.0,2013-04-09 22:17:37,Teresina,Piauí
1,29293,1,GLP 13KG,40.00,Dinheiro,40.00,40.0,2013-05-18 23:12:27,Teresina,Piauí
2,29293,1,GLP 13KG,45.00,Dinheiro,45.00,45.0,2013-06-25 22:21:47,Teresina,Piauí
3,29293,1,GLP 13KG,45.00,Dinheiro,45.00,45.0,2013-08-11 21:51:12,Teresina,Piauí
4,29293,1,GLP 13KG,45.00,Dinheiro,45.00,45.0,2013-10-17 22:57:38,Teresina,Piauí


In [ ]:
dataset['client_id'] = dataset['client_id'].astype('int')
dataset['quantity_product'] = dataset['quantity_product'].astype('float32')
dataset['product'] = dataset['product'].astype('category')
dataset['value_product'] = pd.to_numeric(dataset['value_product'], errors='coerce')
dataset['payments'] = dataset['payments'].astype('category')
dataset['payments_values'] = pd.to_numeric(dataset['payments_values'], errors='coerce')
dataset['total'] = dataset['total'].astype('float32')
dataset['date'] = dataset['date'].astype('datetime64')
dataset['city'] = dataset['city'].astype('category')
dataset['state'] = dataset['state'].astype('category')
dataset.dtypes

client_id                    int64
quantity_product           float32
product                   category
value_product              float64
payments                  category
payments_values            float64
total                      float32
date                datetime64[ns]
city                      category
state                     category
dtype: object

In [ ]:
#dataset_filtered = dataset.query('state=="Piauí" & quantity_product<=2 & product=="GLP 13KG"')
#dataset_filtered = dataset.query('state=="Piauí" & quantity_product<=2 & product=="GLP 13KG"')


 #print ('Quantidade de linhas e colunas', dataset_filtered.shape)
dataset_filtered.head()

,client_id,quantity_product,product,value_product,payments,payments_values,total,date,city,state
0,29293,1.0,GLP 13KG,45.0,Dinheiro,45.0,45.0,2013-04-09 22:17:37,Teresina,Piauí
1,29293,1.0,GLP 13KG,40.0,Dinheiro,40.0,40.0,2013-05-18 23:12:27,Teresina,Piauí
2,29293,1.0,GLP 13KG,45.0,Dinheiro,45.0,45.0,2013-06-25 22:21:47,Teresina,Piauí
3,29293,1.0,GLP 13KG,45.0,Dinheiro,45.0,45.0,2013-08-11 21:51:12,Teresina,Piauí
4,29293,1.0,GLP 13KG,45.0,Dinheiro,45.0,45.0,2013-10-17 22:57:38,Teresina,Piauí


In [ ]:
dataset_filtered.describe()

,client_id,quantity_product,value_product,payments_values,total
count,6.905520e+05,690552.000000,690552.000000,683851.000000,690552.000000
mean,1.373652e+05,2.160715,71.599388,147.917280,152.949677
std,1.832986e+05,12.224780,18.167595,783.611215,800.761292
min,2.929300e+04,1.000000,0.780000,0.780000,0.780000
25%,4.689400e+04,1.000000,60.000000,63.000000,63.000000
50%,6.600200e+04,1.000000,70.000000,75.000000,75.000000
75%,1.389010e+05,1.000000,80.000000,100.000000,100.000000
max,1.243195e+06,4225.000000,950.000000,260901.500000,260901.500000


In [ ]:
dataset_filtered.info()
dataset_filtered['date'] = pd.to_datetime(dataset_filtered['date'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 690552 entries, 0 to 2090615
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   client_id         690552 non-null  int64         
 1   quantity_product  690552 non-null  float32       
 2   product           690552 non-null  category      
 3   value_product     690552 non-null  float64       
 4   payments          690552 non-null  category      
 5   payments_values   683851 non-null  float64       
 6   total             690552 non-null  float32       
 7   date              690552 non-null  datetime64[ns]
 8   city              690552 non-null  category      
 9   state             690552 non-null  category      
dtypes: category(4), datetime64[ns](1), float32(2), float64(2), int64(1)
memory usage: 35.6 MB


<ipython-input-226-ae4c313e25fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_filtered['date'] = pd.to_datetime(dataset_filtered['date'])


In [ ]:
last_order = [-1] * dataset_filtered.shape[0]
for client_id in dataset_filtered['client_id'].unique():
  ordered = (
      dataset_filtered[dataset_filtered['client_id'] == client_id]
      .sort_values('date', axis=0, ascending=True)
  )
  last_date = None
  for date in ordered['date']:
    if (last_date is not None):
      i = dataset_filtered.index[
          (dataset_filtered['client_id'] == client_id) & (dataset_filtered['date'] == date)
      ].to_list()[0]
      order_diff = date - last_date
      last_order[i] = order_diff
    last_date = date

for i in range(len(last_order)):
  if (last_order[i] != -1):
    last_order[i] /= numpy.timedelta64(1, 'D')

dataset_filtered['last_order'] = last_order

TypeError: ignored

In [ ]:
X = dataset_filtered.drop(['last_order', 'date'], axis=1)
y = dataset_filtered['last_order']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
scaler = MinMaxScaler()
columns = dataset_filtered.describe().columns.to_list()
columns.remove('client_id')

X_train_scaled = scaler.fit_transform(X_train)


In [ ]:
dataset_filtered = pd.get_dummies(
    dataset_filtered, prefix='d', columns=['payments', 'product', 'city', 'state']
)

In [ ]:
dataset

## Modelo 1: Regressão linear

In [ ]:
#X = dataset_filtered.drop(['last_order', 'date'], axis=1)
#Y = dataset_filtered['last_order']

In [ ]:
r = LinearRegression()
r = r.fit(X_train_scaled, y_train)

#r.score(X, Y)
X_test_scaled = scaler.transform(X_test)
y_pred = r.predict(X_test_scaled)